In [1]:
from functions import *
import json
from langid.langid import LanguageIdentifier, model
from collections import Counter
import operator
import random
import os
import langid
import seaborn as sns
import matplotlib.colors as mcolors
#https://towardsdatascience.com/visualizing-topic-models-with-scatterpies-and-t-sne-f21f228f7b02
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
#https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/#6.-What-is-the-Dominant-topic-and-its-percentage-contribution-in-each-document

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

# NLTK Stop words
from nltk.corpus import stopwords


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [29]:
#base_path = "/media/ruben/Data Drive/react-data/protest/{}".format("carlo-batch-selection")
base_path = "D:/react-data/protest/{}".format("selection2")
data = pd.read_csv('C:/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_carlo/data-full-dd-full.csv')

In [30]:
# Functions
def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

def remove_stopwords(texts):
    stop_words = stopwords.words('italian')
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def TopicTerms(topic):
    tm = dict(lda_model.show_topics(formatted=False,num_words=10))
    ttt = [i[0] for i in tm[topic]]
    return " | ".join(ttt)

In [57]:
dt = dict()

for lan in list(set(data['language'])):
    ss = data[data['language'] == lan]
    identifiers = list(ss['identifier'])
    sentences = list(ss['sentences'])
    sentences = [" ".join(str(s).split('||')) for s in sentences]
    ss = dict(list(zip(identifiers,sentences)))
    dt.update({lan:ss})
    

id2photo = dict()

for c,i in enumerate(data['photo']):
    ss = data[data['photo'] == i]
    id2photo.update({i:list(set(ss['identifier']))})

In [31]:
# Import Data

dt = dict()
id2photo = dict()

for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    num_iterations = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]
    num_iterations = len(num_iterations)

    start_iter = 1
    range_iter = [str(i) for i in list(range(1,num_iterations + 1))]

    folder_base = os.path.join(base_path,photo,photo)
    
    if photo not in id2photo.keys():
        id2photo.update({photo:list()})
    
    for iteration in range_iter:
        fn = os.path.join(folder_base + "_" +str(iteration),"txt", "parsed_text.json")
        
        with open(fn) as fp:
            pages = json.load(fp)
            
        for identifier,sentences in pages.items():

            id2photo[photo].append(identifier)
            sentences = [s.replace("\n","").lower() for s in sentences]
            sentences = [re.sub(' +', ' ', s) for s in sentences]
            
            language = langid.classify(" ".join(sentences))[0]
            if language not in dt.keys():
                dt.update({language:dict()})
            
            dt[language].update({identifier:" ".join(sentences)})

In [58]:
num_topics = 8

In [66]:
def LDA(num_topics,language):
    sentences_ids = [[k,v] for k,v in dt[language].items()]
    sentences = [i[1] for i in sentences_ids]
    sentences = list(sent_to_words(sentences))
    stop_words = stopwords.words('italian') + stopwords.words('french')
    sentences = remove_stopwords(sentences)
    id2word = corpora.Dictionary(sentences)
    corpus = [id2word.doc2bow(text) for text in sentences]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=num_topics, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=50,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True) #minimum_probability=0.0
    
    refdf = pd.read_csv("C:/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_carlo/descriptive-full.csv")
    refdf = dict(zip(refdf['image'],refdf['pretty_title']))
    
    # Create Dataframe as presented in:
    # https://jeriwieringa.com/2017/06/21/Calculating-and-Visualizing-Topic-Significance-over-Time-Part-1/#topic=0&lambda=1&term=
    df= []

    for c,bow in enumerate(corpus):
        index_pos = c
        doc_text = sentences[c]
        
        photo = [k for k,v in id2photo.items() if sentences_ids[c][0] in v][0]
        
        if len(doc_text) == 0:
            continue
        
        for topic_id,prob in lda_model.get_document_topics(bow):
            df.append([index_pos, topic_id, prob, TopicTerms(topic_id), sentences_ids[c][0], photo])

    df = pd.DataFrame(df,columns=['index_pos', 'topic_id', 'norm_topic_weights', 'topic_words', 'text_id', 'photo'])
    return lda_model,df

def PlotCumulative(lda_model,df,num_topics):
    n_dict = {i[1]:len(df[df['photo'] == i[1]]) for i in list(zip(df['text_id'],df['photo']))}
    dfa = df[['topic_words','photo','norm_topic_weights']].groupby(['topic_words','photo']).sum().reset_index()
    dfa = dfa.pivot(index='photo',columns='topic_words',values='norm_topic_weights').reset_index()
    p = dfa.plot.barh(x='photo',stacked=True,figsize = (20,15),colormap='viridis',width=0.8)
    fig = p.get_figure()
    fig.savefig("/media/ruben/FEF44259F44213F5/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_carlo/topic-document-dist-{}-cumulative-a.png".format(num_topics),dpi=300)
    
def PlotNormalized(lda_model,df,num_topics):
    df2 = pd.DataFrame()

    for ph in list(set(df['photo'])):
        tmp = df[df['photo'] == ph][['topic_words','norm_topic_weights']].groupby('topic_words').sum().sort_values('topic_words').reset_index()
        sum_ = tmp['norm_topic_weights'].sum()
        tmp['norm_topic_weights'] = tmp['norm_topic_weights'] / sum_ * 100
        tmp['photo'] = ph
        df2 = df2.append(tmp)
    df2 = df2.pivot(index="photo",columns='topic_words',values="norm_topic_weights").reset_index()
    df2 = df2.sort_values(list(df2.columns)[1],ascending=True)
    p = df2.plot.barh(x='photo',stacked=True,figsize = (15,15),colormap='viridis',width=0.8)
    plt.legend(list(df2.columns)[1:], loc='upper left', bbox_to_anchor=(1.0, 1.0), fontsize=15)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    fig = p.get_figure()
    fig.savefig("/media/ruben/FEF44259F44213F5/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_carlo/topic-document-dist-{}-normalized-a.png".format(num_topics),
               dpi=300,
               bbox_inches='tight')
    
def PlotWords(lda_model,df,num_topics):
    ## Top words

    topics = lda_model.show_topics(formatted=False)
    data_flat = [w for w_list in sentences for w in w_list]
    counter = Counter(data_flat)
    out = []

    for i, topic in topics:
        for word, weight in topic:
            out.append([word, i , weight, counter[word]])

    df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

    # Plot Word Count and Weights of Topic Keywords
    fig, axes = plt.subplots(3, 2, figsize=(30,20), sharey=True, dpi=300)

    cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
    for i, ax in enumerate(axes.flatten()):

        ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
        ax_twin = ax.twinx()
        ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
        ax.set_ylabel('Word Count', color=cols[i])
        #ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 10000)
        ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
        ax.tick_params(axis='y', left=False)
        ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right',fontsize=22)
        ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

    fig.tight_layout(w_pad=2)    
    fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=22, y=1.05)    
    #plt.savefig('/media/ruben/FEF44259F44213F5/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_carlo/topics-{}-{}-words-a.png'.format('italian',num_topics),dpi=800)
    plt.show()

In [67]:
lda_model,df = LDA(6,language='it')
PlotWords(lda_model,df,6)

KeyError: 4

In [66]:
wh = {3:[1,3,30,10],
          4:[2,2,20,20],
          5:[2,3,20,30],
          6:[2,3,20,30],
          7:[3,3,30,30],
          8:[3,3,30,30]}

for i in range(3,9):
    num_topics = i
    lda_model = LDA(num_topics,'it')
    print('model with {} topics trained'.format(i))
    df = CreateData(lda_model)
    row = wh[i][0]
    col = wh[i][1]
    width = wh[i][2]
    height = wh[i][3]
    PlotWords(lda_model,df,num_topics)

model with 3 topics trained


IndexError: list index out of range